# Setup

In [ ]:
packrat::init(project = "/work/postresearch/Shared/Projects/Farbod/CaseMix")

In [ ]:
packrat::snapshot()

In [ ]:
packrat::status()

In [2]:
setwd('/work/postresearch/Shared/Projects/Farbod/Integration_Intensity')

In [3]:
options(repr.matrix.max.rows=100, repr.matrix.max.cols=300)
options(repr.plot.width = 20, repr.plot.height = 15)
options(width=300)
    
numcores=128

library(tidyverse)
library(parallel)
library(data.table)
library(fst)
library(comorbidity)
library(zeallot)
library(reshape)
library(dtplyr)
library(haven)
library(vroom)
library(dplyr)
#library(icd)
`%!in%` = Negate(`%in%`)

setDTthreads(numcores)

numcores_foreach=8

library(foreach)
library(doMC)
registerDoMC(cores=numcores_foreach)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.0     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose



Attaching package: ‘reshape’


The following object is masked from ‘package:data.table’:

    melt


The following object is masked from ‘package:dplyr’:

    rename


The following objects are masked from ‘package:tidyr’:

    expand, smiths



Attaching package: 

ERROR: Error in library(doMC): there is no package called ‘doMC’


# Finding the diagnosis for each patient

## Carrier files

In [ ]:
carrier_data_2013=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2013.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))
carrier_data_2014=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2014.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))
carrier_data_2015=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2015.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))
carrier_data_2016=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2016.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))
carrier_data_2017=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2017.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))
carrier_data_2018=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2018.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))
carrier_data_2019=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2019.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))
carrier_data_2020=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2020.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))

## Inpatient files

In [ ]:
inpatient_data_2013 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2013.fst",as.data.table = T)
inpatient_data_2014 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2014.fst",as.data.table = T)
inpatient_data_2015 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2015.fst",as.data.table = T)
inpatient_data_2016 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2016.fst",as.data.table = T)
inpatient_data_2017 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2017.fst",as.data.table = T)
inpatient_data_2018 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2018.fst",as.data.table = T)
inpatient_data_2019 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2019.fst",as.data.table = T)
inpatient_data_2020 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2020.fst",as.data.table = T)

## Outpatient files

In [ ]:
outpatient_data_2013 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2013.fst",as.data.table = T)
outpatient_data_2014 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2014.fst",as.data.table = T)
outpatient_data_2015 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2015.fst",as.data.table = T)
outpatient_data_2016 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2016.fst",as.data.table = T)
outpatient_data_2017 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2017.fst",as.data.table = T)
outpatient_data_2018 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2018.fst",as.data.table = T)
outpatient_data_2019 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2019.fst",as.data.table = T)
outpatient_data_2020 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2020.fst",as.data.table = T)

In [ ]:
carrier_test=
carrier_data_2020[1:10,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

In [ ]:
head(carrier_test)

In [ ]:
find_all_diagnosis=function(data){
    data=data[,.(
    DESY_SORT_KEY,
    PRNCPAL_DGNS_CD,
    ICD_DGNS_CD1,
    ICD_DGNS_CD2,
    ICD_DGNS_CD3,
    ICD_DGNS_CD4,
    ICD_DGNS_CD5,
    ICD_DGNS_CD6,
    ICD_DGNS_CD7,
    ICD_DGNS_CD8,
    ICD_DGNS_CD9,
    ICD_DGNS_CD10,
    ICD_DGNS_CD11,
    ICD_DGNS_CD12,
    ICD_DGNS_CD13,
    ICD_DGNS_CD14,
    ICD_DGNS_CD15,
    ICD_DGNS_CD16,
    ICD_DGNS_CD17,
    ICD_DGNS_CD18,
    ICD_DGNS_CD19,
    ICD_DGNS_CD20,
    ICD_DGNS_CD21,
    ICD_DGNS_CD22,
    ICD_DGNS_CD23,
    ICD_DGNS_CD24,
    ICD_DGNS_CD25
    )]
    
    result=melt(data,id.vars = "DESY_SORT_KEY")%>%as.data.table()
    
    result=result[,.(MBI=DESY_SORT_KEY,
                     DIAG=value
                    )]
    
    result=result[nchar(DIAG)>0,]
    
    return(result)
    
    
}

In [ ]:
in_test=find_all_diagnosis(inpatient_data_2020[1:2,])

In [ ]:
out_test=find_all_diagnosis(outpatient_data_2020[1:2,])

In [ ]:
combine_diagnosis=function(carrier,outpatient,inpatient){
    data=rbind(carrier,outpatient,inpatient)
    data=data[order(MBI)]
    return(data)
}

In [ ]:
diag_test=combine_diagnosis(carrier_test,out_test,in_test)

In [ ]:
head(diag_test)

In [ ]:
carrier_2013=
carrier_data_2013[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

carrier_2014=
carrier_data_2014[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

carrier_2015=
carrier_data_2015[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

carrier_2016=
carrier_data_2016[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

carrier_2017=
carrier_data_2017[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

carrier_2018=
carrier_data_2018[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

carrier_2019=
carrier_data_2019[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

carrier_2020=
carrier_data_2020[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

In [ ]:
head(carrier_2013)

In [ ]:
in_2013=find_all_diagnosis(inpatient_data_2013)
in_2014=find_all_diagnosis(inpatient_data_2014)
in_2015=find_all_diagnosis(inpatient_data_2015)
in_2016=find_all_diagnosis(inpatient_data_2016)
in_2017=find_all_diagnosis(inpatient_data_2017)
in_2018=find_all_diagnosis(inpatient_data_2018)
in_2019=find_all_diagnosis(inpatient_data_2019)
in_2020=find_all_diagnosis(inpatient_data_2020)

In [ ]:
head(in_2013)

In [ ]:
out_2013=find_all_diagnosis(outpatient_data_2013)
out_2014=find_all_diagnosis(outpatient_data_2014)
out_2015=find_all_diagnosis(outpatient_data_2015)
out_2016=find_all_diagnosis(outpatient_data_2016)
out_2017=find_all_diagnosis(outpatient_data_2017)
out_2018=find_all_diagnosis(outpatient_data_2018)
out_2019=find_all_diagnosis(outpatient_data_2019)
out_2020=find_all_diagnosis(outpatient_data_2020)

In [ ]:
head(out_2013)

In [ ]:
diag_2013=combine_diagnosis(carrier_2013,out_2013,in_2013)
diag_2014=combine_diagnosis(carrier_2014,out_2014,in_2014)
diag_2015=combine_diagnosis(carrier_2015,out_2015,in_2015)
diag_2016=combine_diagnosis(carrier_2016,out_2016,in_2016)
diag_2017=combine_diagnosis(carrier_2017,out_2017,in_2017)
diag_2018=combine_diagnosis(carrier_2018,out_2018,in_2018)
diag_2019=combine_diagnosis(carrier_2019,out_2019,in_2019)
diag_2020=combine_diagnosis(carrier_2020,out_2020,in_2020)

In [ ]:
head(diag_2013)

In [ ]:
write.csv(diag_2013,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2013.csv")
write.csv(diag_2014,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2014.csv")
write.csv(diag_2015,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2015.csv")
write.csv(diag_2016,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2016.csv")
write.csv(diag_2017,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2017.csv")
write.csv(diag_2018,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2018.csv")
write.csv(diag_2019,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2019.csv")
write.csv(diag_2020,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2020.csv")

# Converting to HCC codes

In [ ]:
diag_2013=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2013.csv", num_threads = numcores)%>% as.data.table()
diag_2014=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2014.csv", num_threads = numcores)%>% as.data.table()
diag_2015=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2015.csv", num_threads = numcores)%>% as.data.table()
diag_2016=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2016.csv", num_threads = numcores)%>% as.data.table()
diag_2017=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2017.csv", num_threads = numcores)%>% as.data.table()
diag_2018=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2018.csv", num_threads = numcores)%>% as.data.table()
diag_2019=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2019.csv", num_threads = numcores)%>% as.data.table()
diag_2020=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2020.csv", num_threads = numcores)%>% as.data.table()

In [ ]:
diag_2013[,date:=as.IDate(paste("2013","-01","-01",sep=""),"%Y-%m-%d")]
diag_2014[,date:=as.IDate(paste("2014","-01","-01",sep=""),"%Y-%m-%d")]
diag_2015[,date:=as.IDate(paste("2015","-01","-01",sep=""),"%Y-%m-%d")]
diag_2016[,date:=as.IDate(paste("2016","-01","-01",sep=""),"%Y-%m-%d")]
diag_2017[,date:=as.IDate(paste("2017","-01","-01",sep=""),"%Y-%m-%d")]
diag_2018[,date:=as.IDate(paste("2018","-01","-01",sep=""),"%Y-%m-%d")]
diag_2019[,date:=as.IDate(paste("2019","-01","-01",sep=""),"%Y-%m-%d")]
diag_2020[,date:=as.IDate(paste("2019","-01","-01",sep=""),"%Y-%m-%d")]

In [ ]:
diag_2013[,MBI:=as.character(MBI)]
diag_2014[,MBI:=as.character(MBI)]
diag_2015[,MBI:=as.character(MBI)]
diag_2016[,MBI:=as.character(MBI)]
diag_2017[,MBI:=as.character(MBI)]
diag_2018[,MBI:=as.character(MBI)]
diag_2019[,MBI:=as.character(MBI)]
diag_2020[,MBI:=as.character(MBI)]

In [ ]:
head(diag_2013)
tail(diag_2020)

In [ ]:
comorbid_hcc(as.data.frame(diag_2013[MBI==100000015,]),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )

In [ ]:
comorbid_hcc(as.data.frame(diag_2015[MBI==100000015,]),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )

In [ ]:
hcc_diag_2013=comorbid_hcc(as.data.frame(diag_2013),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )
hcc_diag_2014=comorbid_hcc(as.data.frame(diag_2014),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )
hcc_diag_2015=comorbid_hcc(as.data.frame(diag_2015),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )
hcc_diag_2016=comorbid_hcc(as.data.frame(diag_2016),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )
hcc_diag_2017=comorbid_hcc(as.data.frame(diag_2017),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )
hcc_diag_2018=comorbid_hcc(as.data.frame(diag_2018),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )
hcc_diag_2019=comorbid_hcc(as.data.frame(diag_2019),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )
hcc_diag_2020=comorbid_hcc(as.data.frame(diag_2020),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )

In [ ]:
hcc_diag_2013=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2013.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2014=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2014.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2015=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2015.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2016=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2016.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2017=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2017.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2018=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2018.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2019=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2019.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2020=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2020.csv",num_threads = numcores)%>%as.data.table()

In [ ]:
hcc_diag_2013=hcc_diag_2013[,.(MBI=as.factor(MBI),hcc)]
hcc_diag_2014=hcc_diag_2014[,.(MBI=as.factor(MBI),hcc)]
hcc_diag_2015=hcc_diag_2015[,.(MBI=as.factor(MBI),hcc)]
hcc_diag_2016=hcc_diag_2016[,.(MBI=as.factor(MBI),hcc)]
hcc_diag_2017=hcc_diag_2017[,.(MBI=as.factor(MBI),hcc)]
hcc_diag_2018=hcc_diag_2018[,.(MBI=as.factor(MBI),hcc)]
hcc_diag_2019=hcc_diag_2019[,.(MBI=as.factor(MBI),hcc)]
hcc_diag_2020=hcc_diag_2020[,.(MBI=as.factor(MBI),hcc)]

In [ ]:
hcc_diag_2013[MBI==100000015]
hcc_diag_2015[MBI==100000015]
hcc_diag_2020[MBI==100000015]

In [ ]:
hcc_diag_2013_wide = dcast(hcc_diag_2013,MBI~hcc,length)
hcc_diag_2014_wide = dcast(hcc_diag_2014,MBI~hcc,length)
hcc_diag_2015_wide = dcast(hcc_diag_2015,MBI~hcc,length)
hcc_diag_2016_wide = dcast(hcc_diag_2016,MBI~hcc,length)
hcc_diag_2017_wide = dcast(hcc_diag_2017,MBI~hcc,length)
hcc_diag_2018_wide = dcast(hcc_diag_2018,MBI~hcc,length)
hcc_diag_2019_wide = dcast(hcc_diag_2019,MBI~hcc,length)
hcc_diag_2020_wide = dcast(hcc_diag_2020,MBI~hcc,length)

In [ ]:
hcc_diag_2013_wide[MBI==100000015]
hcc_diag_2015_wide[MBI==100000015]
hcc_diag_2020_wide[MBI==100000015]

In [ ]:
head(hcc_diag_2013_wide)
head(hcc_diag_2020_wide)

In [ ]:
write.csv(hcc_diag_2013_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2013_wide.csv")
write.csv(hcc_diag_2014_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2014_wide.csv")
write.csv(hcc_diag_2015_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2015_wide.csv")
write.csv(hcc_diag_2016_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2016_wide.csv")
write.csv(hcc_diag_2017_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2017_wide.csv")
write.csv(hcc_diag_2018_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2018_wide.csv")
write.csv(hcc_diag_2019_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2019_wide.csv")
write.csv(hcc_diag_2020_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2020_wide.csv")

## Adding to the stable angina file

In [ ]:
hcc_diag_2013_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2013_wide.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2014_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2014_wide.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2015_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2015_wide.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2016_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2016_wide.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2017_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2017_wide.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2018_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2018_wide.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2019_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2019_wide.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2020_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2020_wide.csv",num_threads = numcores)%>%as.data.table()

In [ ]:
yearly_calculations=vroom("yearly_calculations_with_tin.csv",num_threads = numcores)%>%as.data.table()

In [ ]:
hcc_diag_2013_wide[,year:=2013]
hcc_diag_2014_wide[,year:=2014]
hcc_diag_2015_wide[,year:=2015]
hcc_diag_2016_wide[,year:=2016]
hcc_diag_2017_wide[,year:=2017]
hcc_diag_2018_wide[,year:=2018]
hcc_diag_2019_wide[,year:=2019]
hcc_diag_2020_wide[,year:=2020]

In [ ]:
head(hcc_diag_2013_wide)

In [ ]:
hcc_diag_all_years=
rbind(hcc_diag_2013_wide,
      hcc_diag_2014_wide,
      hcc_diag_2015_wide,
      hcc_diag_2016_wide,
      hcc_diag_2017_wide,
      hcc_diag_2018_wide,
      hcc_diag_2019_wide,
      hcc_diag_2020_wide,fill=T)

In [ ]:
colnames(hcc_diag_all_years)[-2]=paste("hcc",colnames(hcc_diag_all_years)[-2],sep="")

In [ ]:
hcc_diag_all_years[is.na(hcc_diag_all_years)]<-0
hcc_diag_all_years=hcc_diag_all_years[,-1]

In [ ]:
tail(hcc_diag_all_years)

In [ ]:
yearly_calculations[,year_before_diagnosis:=year_first_diagnosed-1]
head(yearly_calculations)

In [ ]:
yearly_calculations_with_diag=left_join(yearly_calculations,hcc_diag_all_years,by=c("DESY_SORT_KEY"="MBI","year_before_diagnosis"="hccyear" ))%>%as.data.table()

In [ ]:
yearly_calculations_with_diag=yearly_calculations_with_diag[,-c("...1","X")]

In [ ]:
head(yearly_calculations_with_diag)

In [ ]:
write.csv(yearly_calculations_with_diag,"yearly_calculations_with_diag.csv")

# Adding last year's part A and part B coverage

In [ ]:
yearly_calculations_with_diag=vroom("yearly_calculations_with_diag.csv",num_threads = numcores)%>%as.data.table()


In [ ]:
yearly_calculations_with_diag[,date_year_before_diagnosis:=(first_diagnosis-365)]
head(yearly_calculations_with_diag)

In [ ]:
twelve_months_after=function(data,colname){

  z=foreach (p = 1:nrow(data), .combine = rbind) %:%
      foreach (a = 0:11, .combine = cbind) %dopar% {
        paste(colname,
              format(data[p,"date_year_before_diagnosis"] %m+% months(a),"%m"),"_",
              format(data[p,"date_year_before_diagnosis"] %m+% months(a),"%Y"),sep="")
      }
  
  y=foreach(p = 1:nrow(data), .combine = rbind) %:%
      foreach (a = 0:11, .combine = cbind) %dopar% {
        as.numeric(format(data[p,"date_year_before_diagnosis"] %m+% months(a),"%Y"))
      }

  return(list(z,y))
}

monthly_characteristics_finder=function(data,colname){
  
  require(dtplyr)
  require(tidyverse)

  z=twelve_months_after(data,colname)[[1]] 
  y=twelve_months_after(data,colname)[[2]] 
  
  result = foreach (p = 1:nrow(data), .combine = rbind) %:%
      foreach (i = 1:12, .combine = cbind) %dopar% {
        if(y[p,i]>=2013){
          a=as.character(z[p,i])
          b=data[p,a]
        }
        else{
          b=NA
        }
        b
      }
  return(result)
}

add_patient_characteristics = function(data){
  
  require(dtplyr)
  require(lubridate)
  require(tidyverse)
  
  result=data%>%
    mutate(   
      year_before_ENTITLEMENT_BUY_IN_IND_sum=
      rowSums(monthly_characteristics_finder(data,"ENTITLEMENT_BUY_IN_IND")==3 
              | monthly_characteristics_finder(data,"ENTITLEMENT_BUY_IN_IND")=="C"
              ,na.rm=T)
          ) %>% 
  as.data.table()
  
  return(result)
}


data_with_part_a_b=add_patient_characteristics(yearly_calculations_with_diag%>%as.data.frame())
head(data_with_part_a_b)

In [ ]:
nrow(data_with_part_a_b[year_before_ENTITLEMENT_BUY_IN_IND_sum!=12])

In [ ]:
write.csv(data_with_part_a_b,"yearly_calculations_with_part_a_b.csv")

# For person file

In [ ]:
mbsf_2013=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2013.csv" , num_threads = numcores) %>% as.data.table
mbsf_2014=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2014.csv" , num_threads = numcores) %>% as.data.table
mbsf_2015=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2015.csv" , num_threads = numcores) %>% as.data.table
mbsf_2016=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2016.csv" , num_threads = numcores) %>% as.data.table
mbsf_2017=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2017.csv" , num_threads = numcores) %>% as.data.table
mbsf_2018=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2018.csv" , num_threads = numcores) %>% as.data.table
mbsf_2019=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2019.csv" , num_threads = numcores) %>% as.data.table
mbsf_2020=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2020.csv" , num_threads = numcores) %>% as.data.table


In [ ]:
mbsf_colnames_2013_2015=c("DESY_SORT_KEY","STATE_CODE","COUNTY_CODE","SEX_CODE","RACE_CODE","AGE","ORIG_REASON_FOR_ENTITLEMENT","CURR_REASON_FOR_ENTITLEMENT","ESRD_INDICATOR","MEDICARE_STATUS_CD","PART_A_TERMINATION_CODE","PART_B_TERMINATION_CODE","ENTITLEMENT_BUY_IN_IND01","ENTITLEMENT_BUY_IN_IND02","ENTITLEMENT_BUY_IN_IND03","ENTITLEMENT_BUY_IN_IND04","ENTITLEMENT_BUY_IN_IND05","ENTITLEMENT_BUY_IN_IND06","ENTITLEMENT_BUY_IN_IND07","ENTITLEMENT_BUY_IN_IND08","ENTITLEMENT_BUY_IN_IND09","ENTITLEMENT_BUY_IN_IND10","ENTITLEMENT_BUY_IN_IND11","ENTITLEMENT_BUY_IN_IND12","HMO_INDICATOR01","HMO_INDICATOR02","HMO_INDICATOR03","HMO_INDICATOR04","HMO_INDICATOR05","HMO_INDICATOR06","HMO_INDICATOR07","HMO_INDICATOR08","HMO_INDICATOR09","HMO_INDICATOR10","HMO_INDICATOR11","HMO_INDICATOR12","HI_COVERAGE","SMI_COVERAGE","HMO_COVERAGE","STATE_BUY_IN_COVERAGE","VALID_DATE_OF_DEATH_SWITCH","DATE_OF_DEATH","REFERENCE_YEAR")

mbsf_colnames_2016_2020=c("DESY_SORT_KEY","REFERENCE_YEAR","SAMPLE_GROUP","STATE_CODE","COUNTY_CODE","STATE_CNTY_FIPS_CD_01","STATE_CNTY_FIPS_CD_02","STATE_CNTY_FIPS_CD_03","STATE_CNTY_FIPS_CD_04","STATE_CNTY_FIPS_CD_05","STATE_CNTY_FIPS_CD_06","STATE_CNTY_FIPS_CD_07","STATE_CNTY_FIPS_CD_08","STATE_CNTY_FIPS_CD_09","STATE_CNTY_FIPS_CD_10","STATE_CNTY_FIPS_CD_11","STATE_CNTY_FIPS_CD_12","SEX_CODE","RACE_CODE","AGE","ORIG_REASON_FOR_ENTITLEMENT","CURR_REASON_FOR_ENTITLEMENT","ESRD_INDICATOR","MDCR_STATUS_CODE_01","MDCR_STATUS_CODE_02","MDCR_STATUS_CODE_03","MDCR_STATUS_CODE_04","MDCR_STATUS_CODE_05","MDCR_STATUS_CODE_06","MDCR_STATUS_CODE_07","MDCR_STATUS_CODE_08","MDCR_STATUS_CODE_09","MDCR_STATUS_CODE_10","MDCR_STATUS_CODE_11","MDCR_STATUS_CODE_12","PART_A_TERMINATION_CODE","PART_B_TERMINATION_CODE","ENTITLEMENT_BUY_IN_IND01","ENTITLEMENT_BUY_IN_IND02","ENTITLEMENT_BUY_IN_IND03","ENTITLEMENT_BUY_IN_IND04","ENTITLEMENT_BUY_IN_IND05","ENTITLEMENT_BUY_IN_IND06","ENTITLEMENT_BUY_IN_IND07","ENTITLEMENT_BUY_IN_IND08","ENTITLEMENT_BUY_IN_IND09","ENTITLEMENT_BUY_IN_IND10","ENTITLEMENT_BUY_IN_IND11","ENTITLEMENT_BUY_IN_IND12","HMO_INDICATOR01","HMO_INDICATOR02","HMO_INDICATOR03","HMO_INDICATOR04","HMO_INDICATOR05","HMO_INDICATOR06","HMO_INDICATOR07","HMO_INDICATOR08","HMO_INDICATOR09","HMO_INDICATOR10","HMO_INDICATOR11","HMO_INDICATOR12","HI_COVERAGE","SMI_COVERAGE","HMO_COVERAGE","STATE_BUY_IN_COVERAGE","VALID_DATE_OF_DEATH_SWITCH","DATE_OF_DEATH","DUAL_STUS_CD_01","DUAL_STUS_CD_02","DUAL_STUS_CD_03","DUAL_STUS_CD_04","DUAL_STUS_CD_05","DUAL_STUS_CD_06","DUAL_STUS_CD_07","DUAL_STUS_CD_08","DUAL_STUS_CD_09","DUAL_STUS_CD_10","DUAL_STUS_CD_11","DUAL_STUS_CD_12")


colnames(mbsf_2013)=mbsf_colnames_2013_2015
colnames(mbsf_2014)=mbsf_colnames_2013_2015
colnames(mbsf_2015)=mbsf_colnames_2013_2015
colnames(mbsf_2016)=mbsf_colnames_2016_2020
colnames(mbsf_2017)=mbsf_colnames_2016_2020
colnames(mbsf_2018)=mbsf_colnames_2016_2020
colnames(mbsf_2019)=mbsf_colnames_2016_2020
colnames(mbsf_2020)=mbsf_colnames_2016_2020

In [ ]:
mbsf_colnames_2013_2015

In [ ]:
head(mbsf_2013)
head(mbsf_2020)

nrow(mbsf_2013)
nrow(mbsf_2020)


In [ ]:
create_person_file_2013_2015=function(mbsf_data){
    
    result=mbsf_data[,
          .(
              MBI=DESY_SORT_KEY,
              SEX=SEX_CODE,
              DOB=as.IDate(paste(
                  as.character(REFERENCE_YEAR-AGE),"-01","-01",sep="")
                       ,"%Y-%m-%d"),
              OREC=ORIG_REASON_FOR_ENTITLEMENT,
              LTIMCAID=0,
              NEMCAID=0
          ),keyby=DESY_SORT_KEY
         ]
    
    result=result[order(MBI)]
    return(result)
}

create_person_file_2016_2020=function(mbsf_data){
    
    result=mbsf_data[,
          .(
              MBI=DESY_SORT_KEY,
              SEX=SEX_CODE,
              DOB=as.IDate(paste(
                  as.character(REFERENCE_YEAR-AGE),"-01","-01",sep="")
                       ,"%Y-%m-%d"),
              OREC=ORIG_REASON_FOR_ENTITLEMENT,
              LTIMCAID=(sum(
                  DUAL_STUS_CD_01 %in% c('02','04','08'),
                  DUAL_STUS_CD_02 %in% c('02','04','08'),
                  DUAL_STUS_CD_03 %in% c('02','04','08'),
                  DUAL_STUS_CD_04 %in% c('02','04','08'),
                  DUAL_STUS_CD_05 %in% c('02','04','08'),
                  DUAL_STUS_CD_06 %in% c('02','04','08'),
                  DUAL_STUS_CD_07 %in% c('02','04','08'),
                  DUAL_STUS_CD_08 %in% c('02','04','08'),
                  DUAL_STUS_CD_09 %in% c('02','04','08'),
                  DUAL_STUS_CD_10 %in% c('02','04','08'),
                  DUAL_STUS_CD_11 %in% c('02','04','08'),
                  DUAL_STUS_CD_12 %in% c('02','04','08'),na.rm=T
              )>0)*1,
              NEMCAID=0
          ),keyby=DESY_SORT_KEY
         ]
    
    result=result[order(MBI)]
    return(result)
}

In [ ]:
person_file_2013=create_person_file_2013_2015(mbsf_2013)
person_file_2014=create_person_file_2013_2015(mbsf_2014)
person_file_2015=create_person_file_2013_2015(mbsf_2015)
person_file_2016=create_person_file_2016_2020(mbsf_2016)
person_file_2017=create_person_file_2016_2020(mbsf_2017)
person_file_2018=create_person_file_2016_2020(mbsf_2018)
person_file_2019=create_person_file_2016_2020(mbsf_2019)
person_file_2020=create_person_file_2016_2020(mbsf_2020)

In [ ]:
write.csv(person_file_2013,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2013.csv")
write.csv(person_file_2014,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2014.csv")
write.csv(person_file_2015,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2015.csv")
write.csv(person_file_2016,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2016.csv")
write.csv(person_file_2017,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2017.csv")
write.csv(person_file_2018,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2018.csv")
write.csv(person_file_2019,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2019.csv")
write.csv(person_file_2020,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2020.csv")

# Recalculating the Charlson scores using the majority of ICD9 or 10

In [ ]:
diag_2013=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2013.csv", num_threads = numcores)%>% as.data.table()
diag_2014=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2014.csv", num_threads = numcores)%>% as.data.table()
diag_2015=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2015.csv", num_threads = numcores)%>% as.data.table()
diag_2016=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2016.csv", num_threads = numcores)%>% as.data.table()
diag_2017=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2017.csv", num_threads = numcores)%>% as.data.table()
diag_2018=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2018.csv", num_threads = numcores)%>% as.data.table()
diag_2019=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2019.csv", num_threads = numcores)%>% as.data.table()
diag_2020=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2020.csv", num_threads = numcores)%>% as.data.table()

In [ ]:
diag_2013[,date:=as.IDate(paste("2013","-01","-01",sep=""),"%Y-%m-%d")]
diag_2014[,date:=as.IDate(paste("2014","-01","-01",sep=""),"%Y-%m-%d")]
diag_2015[,date:=as.IDate(paste("2015","-01","-01",sep=""),"%Y-%m-%d")]
diag_2016[,date:=as.IDate(paste("2016","-01","-01",sep=""),"%Y-%m-%d")]
diag_2017[,date:=as.IDate(paste("2017","-01","-01",sep=""),"%Y-%m-%d")]
diag_2018[,date:=as.IDate(paste("2018","-01","-01",sep=""),"%Y-%m-%d")]
diag_2019[,date:=as.IDate(paste("2019","-01","-01",sep=""),"%Y-%m-%d")]
diag_2020[,date:=as.IDate(paste("2019","-01","-01",sep=""),"%Y-%m-%d")]

In [ ]:
diag_2013[,MBI:=as.character(MBI)]
diag_2014[,MBI:=as.character(MBI)]
diag_2015[,MBI:=as.character(MBI)]
diag_2016[,MBI:=as.character(MBI)]
diag_2017[,MBI:=as.character(MBI)]
diag_2018[,MBI:=as.character(MBI)]
diag_2019[,MBI:=as.character(MBI)]
diag_2020[,MBI:=as.character(MBI)]

In [ ]:
head(diag_2013)
tail(diag_2020)

In [ ]:
comorbidity_icd_9 = comorbidity(
      as.data.table(comorbidity_icd_9),
      id = "DESY_SORT_KEY",
      code = "LINE_ICD_DGNS_CD",
      score = "charlson",
      icd = "icd9",
      assign0 = T
    )

  
comorbidity_icd_10 = comorbidity(
      as.data.table(comorbidity_icd_10),
      id = "DESY_SORT_KEY",
      code = "LINE_ICD_DGNS_CD",
      score = "charlson",
      icd = "icd10",
      assign0 = T
    )

comorbidity_all=rbind(
    comorbidity_icd_9[,c("DESY_SORT_KEY","score","index","wscore","windex")]
    ,comorbidity_icd_10[,c("DESY_SORT_KEY","score","index","wscore","windex")]
  )
  result = left_join(summary_data,
                     comorbidity_all,
                     by="DESY_SORT_KEY",) %>% as.data.table()



summary_with_patient_characteristics_comorbidity=add_comorbidity(data = carrier_data_all_years, summary_data = summary_with_patient_characteristics)
head(summary_with_patient_characteristics_comorbidity)

In [ ]:
score(comorbidity(
      x= as.data.table(diag_2013[MBI==100000015,]),
      id = "MBI",
      code = "DIAG",
      map = "charlson_icd9_quan",
      assign0 = T
    ),assign0 = TRUE,weights="charlson")

In [ ]:
score(comorbidity(
      x= as.data.table(diag_2015[MBI==100000015,]),
      id = "MBI",
      code = "DIAG",
      map = "charlson_icd9_quan",
      assign0 = T
    ),assign0 = TRUE,weights="charlson")

In [ ]:
charlson_2013=
comorbidity(
    x= diag_2013,
    id = "MBI",
    code = "DIAG",
    map = "charlson_icd9_quan",
    assign0 = T
)

In [ ]:
cbind(head(charlson_2013),wscore=score(head(charlson_2013),assign0 = TRUE,weights="charlson"))

In [ ]:
charlson_2013=
comorbidity(
    x= diag_2013,
    id = "MBI",
    code = "DIAG",
    map = "charlson_icd9_quan",
    assign0 = T
)

charlson_2014=
comorbidity(
    x= diag_2014,
    id = "MBI",
    code = "DIAG",
    map = "charlson_icd9_quan",
    assign0 = T
)

charlson_2015=
comorbidity(
    x= diag_2015,
    id = "MBI",
    code = "DIAG",
    map = "charlson_icd9_quan",
    assign0 = T
)

charlson_2016=
comorbidity(
    x= diag_2016,
    id = "MBI",
    code = "DIAG",
    map = "charlson_icd10_quan",
    assign0 = T
)

charlson_2017=
comorbidity(
    x= diag_2017,
    id = "MBI",
    code = "DIAG",
    map = "charlson_icd10_quan",
    assign0 = T
)

charlson_2018=
comorbidity(
    x= diag_2018,
    id = "MBI",
    code = "DIAG",
    map = "charlson_icd10_quan",
    assign0 = T
)

charlson_2019=
comorbidity(
    x= diag_2019,
    id = "MBI",
    code = "DIAG",
    map = "charlson_icd10_quan",
    assign0 = T
)

charlson_2020=
comorbidity(
    x= diag_2020,
    id = "MBI",
    code = "DIAG",
    map = "charlson_icd10_quan",
    assign0 = T
)


In [ ]:
wscore_2013=score(charlson_2013,assign0 = TRUE,weights="charlson")
wscore_2014=score(charlson_2014,assign0 = TRUE,weights="charlson")
wscore_2015=score(charlson_2015,assign0 = TRUE,weights="charlson")
wscore_2016=score(charlson_2016,assign0 = TRUE,weights="charlson")
wscore_2017=score(charlson_2017,assign0 = TRUE,weights="charlson")
wscore_2018=score(charlson_2018,assign0 = TRUE,weights="charlson")
wscore_2019=score(charlson_2019,assign0 = TRUE,weights="charlson")
wscore_2020=score(charlson_2020,assign0 = TRUE,weights="charlson")

In [ ]:
charlson_2013=cbind(charlson_2013,wscore_2013)%>%as.data.table()
charlson_2014=cbind(charlson_2014,wscore_2014)%>%as.data.table()
charlson_2015=cbind(charlson_2015,wscore_2015)%>%as.data.table()
charlson_2016=cbind(charlson_2016,wscore_2016)%>%as.data.table()
charlson_2017=cbind(charlson_2017,wscore_2017)%>%as.data.table()
charlson_2018=cbind(charlson_2018,wscore_2018)%>%as.data.table()
charlson_2019=cbind(charlson_2019,wscore_2019)%>%as.data.table()
charlson_2020=cbind(charlson_2020,wscore_2020)%>%as.data.table()

In [ ]:
colnames(charlson_2013)[19]="wscore"
colnames(charlson_2014)[19]="wscore"
colnames(charlson_2015)[19]="wscore"
colnames(charlson_2016)[19]="wscore"
colnames(charlson_2017)[19]="wscore"
colnames(charlson_2018)[19]="wscore"
colnames(charlson_2019)[19]="wscore"
colnames(charlson_2020)[19]="wscore"

In [ ]:
charlson_2013[,year:=2013]
charlson_2014[,year:=2014]
charlson_2015[,year:=2015]
charlson_2016[,year:=2016]
charlson_2017[,year:=2017]
charlson_2018[,year:=2018]
charlson_2019[,year:=2019]
charlson_2020[,year:=2020]

In [ ]:
head(charlson_2013)
head(charlson_2015)
head(charlson_2020)

In [ ]:
yearly_calculations=vroom("yearly_calculations_with_part_a_b.csv",num_threads = numcores)%>%as.data.table()

In [ ]:
charlson_all_years=
rbind(charlson_2013,
      charlson_2014,
      charlson_2015,
      charlson_2016,
      charlson_2017,
      charlson_2018,
      charlson_2019,
      charlson_2020,fill=T)

In [ ]:
yearly_calculations[,DESY_SORT_KEY:=as.character(DESY_SORT_KEY)]

In [ ]:
colnames(yearly_calculations)[colnames(yearly_calculations)=="wscore"]="wscore_old"

In [ ]:
head(yearly_calculations)

In [ ]:
yearly_calculations_with_charlson=left_join(yearly_calculations,charlson_all_years,by=c("DESY_SORT_KEY"="MBI","year_before_diagnosis"="year"))%>%as.data.table()

In [ ]:
head(yearly_calculations_with_charlson)

In [ ]:
head(yearly_calculations_with_charlson[,.(DESY_SORT_KEY,wscore_old,wscore)])

In [ ]:
yearly_calculations_with_charlson=yearly_calculations_with_charlson[,-c("...1","...2")]

In [ ]:
write.csv(yearly_calculations_with_charlson,"yearly_calculations_with_new_charlson.csv")

## year of diagnosis charlson

In [ ]:
yearly_calculations_with_charlson=left_join(yearly_calculations,charlson_all_years,by=c("DESY_SORT_KEY"="MBI","year_first_diagnosed"="year"))%>%as.data.table()

In [ ]:
head(yearly_calculations_with_charlson)

In [ ]:
head(yearly_calculations_with_charlson[,.(DESY_SORT_KEY,wscore_old,wscore)],20)

In [ ]:
yearly_calculations_with_charlson=yearly_calculations_with_charlson[,-c("...1","...2")]

In [ ]:
write.csv(yearly_calculations_with_charlson,"yearly_calculations_with_new_charlson_year_of_diagnosis.csv")

# Testing

In [4]:
yearly_data=read.csv("yearly_calculations_with_new_charlson_year_of_diagnosis.csv")%>%as.data.table()

In [16]:
head(yearly_data)

X,DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,MI_date,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,

In [13]:
# find the column names that contant "year_before"
grep("year_before", colnames(yearly_data), value = T)

[1] "most_common_physician_prp_in_year_before_diagnosis_05_non_exclusive_HCPCS"                   "most_common_physician_prp_in_year_before_diagnosis_03_non_exclusive_HCPCS"                   "most_common_physician_avg_in_year_before_diagnosis_non_exclusive_HCPCS"                     
 [4] "most_common_primary_care_physician_prp_in_year_before_diagnosis_05_non_exclusive_HCPCS"      "most_common_primary_care_physician_prp_in_year_before_diagnosis_03_non_exclusive_HCPCS"      "most_common_primary_care_physician_avg_in_year_before_diagnosis_non_exclusive_HCPCS"        
 [7] "most_common_cardiologist_prp_in_year_before_diagnosis_05_non_exclusive_HCPCS"                "most_common_cardiologist_prp_in_year_before_diagnosis_03_non_exclusive_HCPCS"                "most_common_cardiologist_avg_in_year_before_diagnosis_non_exclusive_HCPCS"                  
[10] "diagnosing_doc_prp_in_year_before_diagnosis_05_non_exclusive_HCPCS"                          "diagnosing_doc_prp_in_year_before_diagnosis_03_non_exclusive_HCPCS"                          "diagnosing_doc_avg_in_year_before_diagnosis_non_exclusive_HCPCS"                            
[13] "catheterization_doc_prp_in_year_before_diagnosis_05_non_exclusive_HCPCS"                     "catheterization_doc_prp_in_year_before_diagnosis_03_non_exclusive_HCPCS"                     "catheterization_doc_avg_in_year_before_diagnosis_non_exclusive_HCPCS"                       
[16] "angioplasty_doc_prp_in_year_before_diagnosis_05_non_exclusive_HCPCS"                         "angioplasty_doc_prp_in_year_before_diagnosis_03_non_exclusive_HCPCS"                         "angioplasty_doc_avg_in_year_before_diagnosis_non_exclusive_HCPCS"                           
[19] "most_common_physician_2013_prp_in_year_before_diagnosis_05_non_exclusive_HCPCS"              "most_common_physician_2013_prp_in_year_before_diagnosis_03_non_exclusive_HCPCS"              "most_common_physician_2013_avg_in_year_before_diagnosis_non_exclusive_HCPCS"                
[22] "most_common_primary_care_physician_2013_prp_in_year_before_diagnosis_05_non_exclusive_HCPCS" "most_common_primary_care_physician_2013_prp_in_year_before_diagnosis_03_non_exclusive_HCPCS" "most_common_primary_care_physician_2013_avg_in_year_before_diagnosis_non_exclusive_HCPCS"   
[25] "most_common_cardiologist_2013_prp_in_year_before_diagnosis_05_non_exclusive_HCPCS"           "most_common_cardiologist_2013_prp_in_year_before_diagnosis_03_non_exclusive_HCPCS"           "most_common_cardiologist_2013_avg_in_year_before_diagnosis_non_exclusive_HCPCS"             
[28] "year_before_diagnosis"                                                                       "date_year_before_diagnosis"                                                                  "year_before_ENTITLEMENT_BUY_IN_IND_sum"

In [25]:
yearly_data[
    unique(most_common_cardiologist_PRF_PHYSN_NPI),
    mean(most_common_cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS >= 0.75,
        na.rm = TRUE
    ),
    keyby = year
]

Warning message in `[.data.table`(yearly_data, unique(most_common_cardiologist_PRF_PHYSN_NPI), :
“NAs introduced by coercion to integer range”


ERROR: Error in as.vector(x, "list"): cannot coerce type 'closure' to vector of type 'list'


In [33]:
yearly_data[
    unique(most_common_cardiologist_PRF_PHYSN_NPI),
    mean(most_common_cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS >= 0.75),
    by = year_first_diagnosed
]

Warning message in `[.data.table`(yearly_data, unique(most_common_cardiologist_PRF_PHYSN_NPI), :
“NAs introduced by coercion to integer range”


year_first_diagnosed,V1
<int>,<dbl>
NA,NA
